# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [ ]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

--2020-06-19 20:23:43--  https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 91.198.174.208, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|91.198.174.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223725 (218K) [image/jpeg]
Saving to: ‘La_noche_estrellada1.jpg.2’

La_noche_estrellada 100%[===================>] 218.48K  --.-KB/s    in 0.02s   

2020-06-19 20:23:43 (13.5 MB/s) - ‘La_noche_estrellada1.jpg.2’ saved [223725/223725]

--2020-06-19 20:23:45--  https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 91.198.174.208, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|91.198.174.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153015 (149K) 

In [ ]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

#base_image_path = Path("/content/775px-Neckarfront_Tübingen_Mai_2017.jpg")
#style_reference_image_path = Path("/content/La_noche_estrellada1.jpg")
#Pregunta 9:
base_image_path = Path("/content/drive/My Drive/Deep Learning/TP Final/Anto.jpeg")
style_reference_image_path = Path("/content/drive/My Drive/Deep Learning/TP Final/Campo_tulipanes.jpeg")

#result_prefix = Path("/content/output")
#result_prefix = Path("/content/drive/My Drive/Deep Learning/TP Final/Imagenes/La_Noche_estrellada")
result_prefix = Path("/content/drive/My Drive/Deep Learning/TP Final/Imagenes/Anto")
iterations = 100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:


El objetivo de la transferencia de estilo consiste en generar una imagen con el mismo "contenido" que una imagen base, pero con el "estilo" de una imagen diferente.

Los parámetros definidos corresponden a:

- El peso que se le otorga a la loss de estilo (style weight), donde defino cuan presente quiero que esté el estilo en la imagen final.
-El peso que se le otorga a la loss de imagen base (Content_weight), un mayor peso hará que la imagen resultante tenga mayor similitud a la imagen original, y encontremos la imagen base más nitida.Y por el contrario, un menor peso en el contenido, hará que la imagen resultante presente más del estilo y el contenido será más difuso.
-En cuanto a total_variation_weight corresponde a la contribución de cada capa en la loss total, dentro de la loss de estilo.

La idea es lograr minimizar la loss de ambos conceptos para lograr captar el estilo y el contenido de la fotografia. 


In [ ]:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

In [ ]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta:
La funcion load_img carga la imagen y redimensiona la imagen al tamaño especificado. 
El siguiente paso convierte la imagen a un array de numpy (img_to_array) y cambia sus dimensiones con (np_expand_dim). 

La funcion model_predict requiere que la entrada sea un array de 4 dimensiones.
 
El paso antes de la prediccion es la normalizacion de datos (preprocess_input), esta función centra en cero los datos usando la media de los valores de los canales de las imágenes del conjunto de entrenamiento.

In [ ]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta:
En este caso lo que se hace es regresar la imagen a tres dimensiones. Y tambien pasa el formato de la imagen de BRG a RGB. En el primer formato el azul tiene mas relevancia y el rojo menos, y en el formato RGB el rojo tiene mas peso y el azul menos. En el paso anterior pasamos la imagen a 4 dimensiones ya que es para entrenar el modelo necesitamos tener estas cuatro dimensiones, ahora lo que hacemos es volver a tener 3 dimensiones.

In [ ]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [ ]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [ ]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [ ]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

80142336/80134624 [==============================] - 3s 0us/step
Model loaded.


# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?
- ¿Por qué se permutan las dimensiones de x?

La matriz de Gram representa el producto de los vectores o filtros. Cuanto mayor similitud haya entre los vectores (correlacion), el producto va ser mayor y la matriz sera larga.
Es decir, la matriz Gram muestra la correlacion entre los distintos filtros de una capa. Se utiliza para calcular la contribucion de cada capa a la loss de estilo. 

¿Por qué se permutan las dimensiones de x?
 
Se permutan las dimensiones x para que quede en formato RGB.

In [ ]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:
- Style loss es la diferencia de correlación presente entre las características de las capas calculadas por la imagen generada y la imagen de estilo.
- Content_loss captura el error cuadrático medio entre las activaciones producidas por la imagen generada y la imagen de contenido.
- Total_variation_loss es la cambinacion de ambas loss, donde "a" y "b" son hiperparametros definidos que permiten ponderar el peso de estilo o contenido que estaremos inyectando en la imagen generada.

In [ ]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [ ]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [ ]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [ ]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [ ]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:

La función fmin_l_bfgs_b utiliza el algoritmo L-BFGS-B. Se trata de un método que hace un uso limitado de la memoria (usa mucha menos memoria que otros algoritmos para el mismo problema). Permite obtener el mínimo de una función. Únicamente necesita la función y su gradiente.
Se busca minimizar la loss de estilo.

In [ ]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [ ]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [ ]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 8132898000.0
Image saved as /content/drive/My Drive/Deep Learning/TP Final/Imagenes/Anto/output_at_iteration_0.png
Iteration 0 completed in 14s
Start of iteration 1
Current loss value: 4447154700.0
Image saved as /content/drive/My Drive/Deep Learning/TP Final/Imagenes/Anto/output_at_iteration_1.png
Iteration 1 completed in 5s
Start of iteration 2
Current loss value: 3416171000.0
Image saved as /content/drive/My Drive/Deep Learning/TP Final/Imagenes/Anto/output_at_iteration_2.png
Iteration 2 completed in 5s
Start of iteration 3
Current loss value: 2907605000.0
Image saved as /content/drive/My Drive/Deep Learning/TP Final/Imagenes/Anto/output_at_iteration_3.png
Iteration 3 completed in 5s
Start of iteration 4
Current loss value: 2539147800.0
Image saved as /content/drive/My Drive/Deep Learning/TP Final/Imagenes/Anto/output_at_iteration_4.png
Iteration 4 completed in 5s
Start of iteration 5
Current loss value: 2299460600.0
Image saved as /content/d

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)
Respuesta:

Realice tres pruebas. 
- En la primer prueba utilice los pesos que ya estaban definidos:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1
El resultado fue la imagen output_at_iteration_99_Estilo.png donde prevalece mucho el estilo, dado que se le asigna un peso mucho mayor. El contenido, es decir el paisaje en esta nueva imagen posee poca nitidez.
- En la segunda prueba invertí los pesos respecto al contenido y al estilo:
total_variation_weight = 0.1
style_weight = 1
content_weight = 10
El resultado fue la imagen output_at_iteration_99_contenido.png, en este caso el resultado, es una imagen con el contenido más nítido y mayor presencia, y por el contrario el estilo se observa con menor presencia.
- La tercer prueba que realicé fue ponderar de la misma forma contenido y estilo. Los parámetros que utilice son:
total_variation_weight = 0.5
style_weight = 10
content_weight = 10
En este caso el resultado generado es la imagen Output_at_iteration_99_50_cont_&_50_estilo.png, la misma es una combinación de ambas imágenes en partes iguales.

# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta:

Realice una prueba con una imagen de mi hija Antonella y la imagen de la pintura Campos de Tuplipanes de Vincent Van Gogh. El contenido es la imagen Anto.jpeg  y el estilo es la imagen Campo_Tulipanes.jpeg.
Los parametros fijados son:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1
El resultado es la imagen de mi hija, pero con estilo de la pintura de Vincent Van Gogh. Se observa como los colores que se utilizan en el cuadro estan presentes en la nueva imagen y el contenido es el rostro de mi hija.